
# <center> Data Preparation </center>
<HR>

<div style='text-align: justify'>
In this notebook, we will manipulate raw data of covid19 as per the requirement of our case study. We want to perform the statistical test, ANOVA on our formulated hypothesis such as difference in number of covid19 cases across the different regions are significant.
So, we will aggregate the data into four different regions based on the continents and consider the 20 countries from each regions for our case study. Data has been collected on 9th May 2020 from Johns Hopkins University.
</div>

Data Source: [Covid19 data Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19)

[]()

Follow [notebook case_study](case_study.ipynb) for statistical test

In [1]:
import pandas as pd
import pycountry_convert as pc

**Load the data into the memory**

In [2]:
df = pd.read_csv('data/covid_data_05-09-2020.csv')

In [3]:
df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-05-10 02:32:30,34.223334,-82.461707,34,0,0,34,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-05-10 02:32:30,30.295065,-92.414197,151,11,0,140,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-05-10 02:32:30,37.767072,-75.632346,496,7,0,489,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-05-10 02:32:30,43.452658,-116.241552,731,19,0,712,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-05-10 02:32:30,41.330756,-94.471059,3,0,0,3,"Adair, Iowa, US"


**Aggregate the data by Country_Region**

Aggregate the data by Country_Region and select required columns for our study such as confirmed covid cases, death etc.

In [4]:
df = df.groupby('Country_Region')[['Confirmed', 
                                   'Deaths', 'Recovered', 'Active']].sum()
df = df.reset_index()
# Rename column Country_Region to Country
df = df.rename(columns={'Country_Region': 'Country'})

In [5]:
df.head()

,Country,Confirmed,Deaths,Recovered,Active
0,Afghanistan,4042,115,502,3416
1,Albania,856,31,627,198
2,Algeria,5558,494,2546,2518
3,Andorra,754,48,545,161
4,Angola,43,2,13,28


**Missing Values in data**

In [6]:
df.isnull().sum()

Country      0
Confirmed    0
Deaths       0
Recovered    0
Active       0
dtype: int64

**Categories the countries into continents**

In [7]:
def country_to_continent(country_name:str) -> str:
    """find continent name of the country"""
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
    except:
        #if country_name is invalid 
        continent_name = pd.NA
    return continent_name

In [8]:
# Change 'US' to 'United States' in Country field Otherwise country name is considered invalid
df.loc[df.Country=='US', 'Country'] = 'United States'
# find continent name for the countries
df['Continent'] = df.Country.map(country_to_continent)

**Categories the continents into the following four regions for our case study:**
    
    AFRO: Africa
    
    AMER: North America and South America
    
    EURO: Europe
    
    OCEA: Asia and Oceania

In [9]:
df['Region'] = df.Continent.map({'Africa': 'AFRO', 
                                 'North America': 'AMR',
                                 'South America': 'AMR',
                                 'Europe': "EURO",
                                 'Asia': 'OCEA',
                                 'Oceania': 'OCEA'})

**Consider 20 top most affected countries from each region for the study**

In [10]:
df = df.groupby(['Region']) \
    .apply(lambda x: x.sort_values(['Confirmed'], ascending=False)) \
    .reset_index(drop=True) \
    .groupby(['Region']) \
    .head(20)

In [11]:
df.tail()

,Country,Confirmed,Deaths,Recovered,Active,Continent,Region
143,Australia,6939,97,6141,701,Oceania,OCEA
144,Malaysia,6589,108,4929,1552,Asia,OCEA
145,Kazakhstan,4975,31,1776,3168,Asia,OCEA
146,Bahrain,4774,8,2055,2711,Asia,OCEA
147,Afghanistan,4042,115,502,3416,Asia,OCEA


**Save the preprocessed data**

In [12]:
df.to_csv('data/covid19_preprocessed_data.csv', index=False)